In [ ]:
#######################################################
########### This is the codes for ML-JI ###############
#######################################################

### Languages
#The python version is python37
#The tensorflow vesion is tensorflow1.14
#Please check and install the other libraries 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os, logging, pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import signal
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers, regularizers
## Ignore all the warnings
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_WARNINGS'] = '0'

plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
#First, for LSTM data

In [ ]:
#Data preprocessing functions.
#read, wrap and normalize the data


def get_station_data(fname):
    """
    Obtain the pandas dataframe from dataset.

    """
    dataset = pd.read_csv(fname)
    dataset["date"] = pd.to_datetime(dataset['date'], dayfirst=True,errors='ignore')
    #dataset = dataset.replace(-99, np.NaN)
    dataset = dataset.set_index("date")
    dataset = dataset.dropna()
    
    # This date range is a fake date that helps better arrange the data.
    # This can be changed with orders
    dataset = dataset.loc[pd.date_range(start='1/1/1980', end='28/10/2009')]

    return dataset


def get_wrapped_data(dataset, wrap_length=214):
    """
    Wrap the data for the shape requirement of LSTM.

    Parameters
    ----------
    dataset: the pandas dataframe obtained from the function get_station_data().
    wrap_length: the number of time steps to be considered for the LSTM layer.

    Returns
    ----------
    data_x_dict: the input dictionary whose key is the date and value is the corresponding wrapped input matrix of each sample.
    data_y_dict: the output dictionary whose key is the date and value is the corresponding target of each sample.
    """
    data_x_dict, data_y_dict = {}, {}

    for date_i in tqdm(dataset.index, desc=f'Prep aring data with wrap length = {wrap_length}'):
        try:
            
            #adjusting input and output features by choosing the columns here, also in split_train_test
            
            data_x = dataset.loc[pd.date_range(end=date_i,
                                               periods=wrap_length + 1,
                                               freq="d")[:-1], ["Prep","Prep_5","Prep_10","Prep_15"],
                                ].to_numpy(dtype='float16')
            data_y = dataset.loc[pd.date_range(end=date_i,
                                               periods=wrap_length + 1,
                                               freq="d")[-1:], ["w1","w2","w3","w4","w5","w6","w7","w8_1","w8_2","w8_3","w8_4","w9","w10","w11","w12_1","w12_2","w12_3","w12_4"],
                                ].to_numpy(dtype='float16')

            data_x_dict[date_i] = data_x
            data_y_dict[date_i] = data_y
        except KeyError:
            continue

    return data_x_dict, data_y_dict


def split_train_test(dataset, data_x_dict, data_y_dict, frac=0.8, random_state=100, scale=True):
    """
    Randomly split the dataset for training and testing.

    Parameters
    ----------
    dataset: the pandas dataframe obtained from the function get_station_data().
    data_x_dict: the input dictionary obtained from the function get_wrapped_data().
    data_y_dict: the output dictionary obtained from the function get_wrapped_data().
    frac: the fraction of samples to be trained.
    random_state: the random seed (default: 100).
    scale: [bool] whether scale the split dataset by the mean and std values of the training data (default: True).

    Returns
    ----------
    train_dates: the dates of the picked training data.
    test_dates: the dates of the picked testing data.
    train_x: the (scaled) inputs for training.
    train_y: the (scaled) outputs for training.
    test_x: the (scaled) inputs for testing.
    test_y: the (scaled) outputs for testing.
    scale_params: the mean and std values of the training data (available when scale is True)
    """
    train_dates = (dataset.loc[data_x_dict.keys()].sample(frac=frac, random_state=random_state).index)
    test_dates  = dataset.loc[data_x_dict.keys()].drop(train_dates).index

    train_x = np.stack([data_x_dict.get(i) for i in train_dates.to_list()])
    train_y = np.stack([data_y_dict.get(i) for i in train_dates.to_list()])
    test_x  = np.stack([data_x_dict.get(i) for i in test_dates.to_list()])
    test_y  = np.stack([data_y_dict.get(i) for i in test_dates.to_list()])

    scale_params = {"train_x_mean": 0, "train_x_std": 1, "train_y_mean": 0, "train_y_std": 1}

    if scale is False:
        return train_dates, test_dates, train_x, train_y, test_x, test_y, scale_params
    else:
        scale_params["train_x_mean"] = (dataset.loc[train_dates, ["Prep","Prep_5","Prep_10","Prep_15"]].mean().values)
        scale_params["train_x_std"]  = (dataset.loc[train_dates, ["Prep","Prep_5","Prep_10","Prep_15"]].std().values)
        scale_params["train_y_mean"] = dataset.loc[train_dates, ["w1","w2","w3","w4","w5","w6","w7","w8_1","w8_2","w8_3","w8_4","w9","w10","w11","w12_1","w12_2","w12_3","w12_4"]].mean().values
        scale_params["train_y_std"]  = dataset.loc[train_dates, ["w1","w2","w3","w4","w5","w6","w7","w8_1","w8_2","w8_3","w8_4","w9","w10","w11","w12_1","w12_2","w12_3","w12_4"]].std().values

        train_x = (train_x - scale_params["train_x_mean"][None, None, :]) / scale_params["train_x_std"][None, None, :]
        train_y = (train_y - scale_params["train_y_mean"][None, :]) / scale_params["train_y_std"][None, :]
        test_x  = (test_x - scale_params["train_x_mean"][None, None, :]) / scale_params["train_x_std"][None, None, :]
        test_y  = (test_y - scale_params["train_y_mean"][None, :]) / scale_params["train_y_std"][None, :]

        return train_dates, test_dates, train_x, train_y, test_x, test_y, scale_params

In [ ]:
WORKING_PATH  = r"C:\Users\e0427809\Desktop\Try\well_50"

####################
#   Basin set up   #
####################
STATION_ID = 'merged_wells_50_CNN' # USGS code used in the MOPEX dataset

####################
#  Hyperparameters #
####################
RANDOM_SEED   = 100        
WRAP_LENGTH   = 214        # Timestep of the LSTM model
TRAIN_FRAC    = 0.8        # The fraction of spliting traning and testing dataset

LEARNING_RATE = 0.03
EPOCH_NUMBER  = 300 # from 200 to 500

In [ ]:
#all the input and output should be in the same file here.
read_path = os.path.join(WORKING_PATH, f'{STATION_ID}.csv')
#you need to create a folder "data" to save the wrapped and normalized data 
data_path  = os.path.join(WORKING_PATH, 'data', f'{STATION_ID}_data.pickle')
#you need to create a folder "results" and a sub-folder "model" to save the trained ML model. 
model_path = os.path.join(WORKING_PATH, 'results', 'model', f'{STATION_ID}_{RANDOM_SEED}_keras.h5')

In [ ]:
hydrodata = get_station_data(fname=read_path)

In [ ]:
if os.path.exists(data_path):
    with open(data_path, 'rb') as f:
        data_x_dict, data_y_dict = pickle.load(f)
else:
    data_x_dict, data_y_dict = get_wrapped_data(dataset=hydrodata,  wrap_length=WRAP_LENGTH)
    with open(data_path, 'wb') as f:
        pickle.dump([data_x_dict, data_y_dict], f)

In [ ]:
split_results = split_train_test(dataset=hydrodata, 
                                       data_x_dict=data_x_dict, 
                                       data_y_dict=data_y_dict, 
                                       frac=TRAIN_FRAC, 
                                       random_state=RANDOM_SEED, 
                                       scale=True)

train_dates, test_dates, x_train, y_train, x_test, y_test, scale_params = split_results

print(f'The shape of x_train, y_train after wrapping by {WRAP_LENGTH} days are {x_train.shape}, {y_train.shape}')
print(f'The shape of x_test, y_test after wrapping by {WRAP_LENGTH} days are   {x_test.shape}, {y_test.shape}')

In [ ]:
split_results = split_train_test(dataset=hydrodata, 
                                       data_x_dict=data_x_dict, 
                                       data_y_dict=data_y_dict, 
                                       frac=TRAIN_FRAC, 
                                       random_state=RANDOM_SEED, 
                                       scale=True)

train_dates, test_dates, x_train, y_train, x_test, y_test, scale_params = split_results

print(f'The shape of x_train, y_train after wrapping by {WRAP_LENGTH} days are {x_train.shape}, {y_train.shape}')
print(f'The shape of x_test, y_test after wrapping by {WRAP_LENGTH} days are   {x_test.shape}, {y_test.shape}')


In [ ]:
#Now, for the Spatial data

In [ ]:
df = pd.read_csv(r'C:\Users\e0427809\Desktop\Try\Spatial\spatial_features.csv', header=None, index_col=False)
df_normalized = pd.DataFrame()

# Iterate over each column for Z-score normalization
for column in df.columns:
    data = df[column]
    mean = np.mean(data)
    std = np.std(data)
    normalized_data = (data - mean) / std
    df_normalized[column] = normalized_data

df_normalized.to_csv(r'C:\Users\e0427809\Desktop\Try\Spatial\normed.csv', index=False, header=False)

In [ ]:
sp = pd.read_csv(r'C:\Users\e0427809\Desktop\Try\Spatial\normed.csv', header=None, index_col=False)
#Make sure the data is nomlized.
data = sp.values

# Reshape the data to fit the input requirements of the CNN model.
data_cnn_reshape = data.reshape(18, 32, 1)

#repeat the data to make sure it Consistent with LSTM data so that the model can run.
#repeating numbers should be changed regarding the shapes of data for LSTM model.
x_train_cnn = np.repeat(data_cnn_reshape, 8544, axis=0)
data_cnn = np.reshape(x_train_cnn, (8544, 18, 32, 1))
x_test_cnn = np.repeat(data_cnn_reshape, 2136, axis=0)
test_cnn = np.reshape(x_test_cnn, (2136, 18, 32, 1))

In [ ]:
#There are loss functions for choosing.
#Not sure about the issues with weighted_rmse_1 and weighted_rmse_2

def mse_loss(y_true, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)  
    return tf.reduce_mean(mse)
def mean_r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)  
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)  
    r2 = 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())  
    return tf.reduce_mean(r2) 

def nse_loss(y_true, y_pred):
    epsilon = 1e-7  
    y_pred_smoothed = tf.where(tf.equal(y_pred, 0), y_pred + epsilon, y_pred)
    numerator = tf.reduce_sum(tf.square(y_true - y_pred_smoothed), axis=0)
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
    nse = 1 - (numerator / denominator)
    return tf.reduce_mean(1-nse)


def mse_loss(y_true, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)  
    return tf.reduce_mean(mse)


def weighted_rmse_1(y_true, y_pred):
    threshold = 4.0  
    weight_high = 3.0  
    weight_low = 0.5 

    weights = K.switch(y_true > threshold, K.ones_like(y_true) * weight_high, K.ones_like(y_true) * weight_low)  

    mse = K.mean(K.square(y_true - y_pred), axis=-1)
    weighted_mse = mse * weights  

    return K.mean(K.sqrt(weighted_mse))

def weighted_rmse_2(y_true, y_pred):
    threshold = 4.0  
    weight_high = 3.0  
    weight_low = 0.5 
    
    weights = K.ones_like(y_true) * weight_low  # Initialize weights as weight_low for all elements
    
    high_indices = tf.cast(y_true > threshold, dtype=K.floatx())  # Create a tensor of 1.0 for high indices and 0.0 for others
    weights = weights * (1.0 - high_indices) + high_indices * weight_high  # Element-wise multiplication and addition
    
    mse = K.mean(K.square(y_true - y_pred), axis=-1)
    weighted_mse = mse * weights
    
    return K.mean(K.sqrt(weighted_mse))


def custom_loss(y_true, y_pred):
    # Filter out values where y_true is less than or equal to 4
    mask = tf.greater(y_true, 4.0)
    filtered_true = tf.boolean_mask(y_true, mask)
    filtered_pred = tf.boolean_mask(y_pred, mask)
    
    # Check if filtered tensors are empty
    if tf.size(filtered_true) == 0:
        return 0.0  # Return 0 if no valid values are available
    
    # Calculate RMSE only for filtered values
    mse = K.mean(K.square(filtered_true - filtered_pred), axis=-1)
    rmse = K.sqrt(mse)
    
    return rmse




def kge_loss(y_true, y_pred):
    # Split y_pred into observed and modeled data
    observed_data = y_true
    modeled_data = y_pred

    # Compute the correlation coefficient between observed and modeled data
    observed_mean = tf.reduce_mean(observed_data)
    modeled_mean = tf.reduce_mean(modeled_data)
    observed_std = tf.math.reduce_std(observed_data)
    modeled_std = tf.math.reduce_std(modeled_data)

    centered_observed = observed_data - observed_mean
    centered_modeled = modeled_data - modeled_mean

    r = tf.reduce_mean(centered_observed * centered_modeled) / (observed_std * modeled_std)

    # Compute the mean ratio (β) and standard deviation ratio (γ)
    mean_ratio = modeled_mean / observed_mean
    std_ratio = modeled_std / observed_std

    # Compute the KGE
    kge = 1 - tf.sqrt((r - 1) ** 2 + (mean_ratio - 1) ** 2 + (std_ratio - 1) ** 2)

    return -kge  # Negate the KGE to be minimized as a loss

In [ ]:
##Now set up the Parallel LSTM+CNN model

In [ ]:
input_cnn = layers.Input(shape=(18, 32, 1), name='input_cnn')
cnn = layers.Conv2D(filters=3, kernel_size=(3,3), activation='relu')(input_cnn)
cnn = layers.MaxPooling2D(pool_size=(3, 3))(cnn)
cnn = layers.Flatten()(cnn)
dense_cnn = layers.Dense(units=18, activation='relu', name='dense_cnn')(cnn)

input_lstm = layers.Input(x_train.shape[1:], name='input_lstm')
lstm = layers.LSTM(units=60, name='lstm',
                   kernel_regularizer=regularizers.l2(0.001),
                   recurrent_regularizer=regularizers.l2(0.001))(input_lstm)

concatenated = layers.concatenate([dense_cnn, lstm])

output = layers.Dense(units=18, name='dense_output', activation='linear', use_bias=False, 
                      kernel_regularizer=regularizers.l2(0.001))(concatenated)
output = layers.Reshape(target_shape=(1, 18))(output)
model = models.Model(inputs=[input_cnn, input_lstm], outputs=output)
model.summary()

In [ ]:
## Run the model

In [ ]:
es     = callbacks.EarlyStopping(monitor='val_R2', mode='max', verbose=1, patience=30, 
                                 min_delta=0.01, restore_best_weights=True)
reduce = callbacks.ReduceLROnPlateau(monitor='val_R2', factor=0.5, patience=15, verbose=1, 
                                     mode='max', min_delta=0.01, cooldown=0, min_lr=LEARNING_RATE / 100)
tnan   = callbacks.TerminateOnNaN()

model.compile(loss = mse_loss, metrics=[mean_r2], optimizer=optimizers.Adam(lr=LEARNING_RATE))
model.fit([data_cnn,x_train], y_train, epochs=EPOCH_NUMBER, batch_size=214, validation_split=0.2, 
          callbacks=[es, reduce, tnan])
model.save(model_path)

In [ ]:
pred_train = model.predict([data_cnn,x_train], batch_size=214)
pred_test  = model.predict([test_cnn,x_test], batch_size=214)

In [ ]:
pr_train = pred_train * scale_params['train_y_std'] + scale_params['train_y_mean']
pr_test = pred_test  * scale_params['train_y_std'] + scale_params['train_y_mean']

pr_train.shape

In [ ]:
#evaluating index

def cal_nse(obs, sim):

    # compute numerator and denominator
    numerator   = np.nansum((obs - sim)**2)
    denominator = np.nansum((obs - np.nanmean(obs))**2)
    # compute coefficient
    return 1 - (numerator / denominator)

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


def calculate_rmse(observed, predicted):
    mse = np.mean((observed - predicted)**2)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
hydrodata.loc[train_dates, ['flow_pred_w1']] = pr_train[:,0,0].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w2']] = pr_train[:,0,1].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w3']] = pr_train[:,0,2].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w4']] = pr_train[:,0,3].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w5']] = pr_train[:,0,4].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w6']] = pr_train[:,0,5].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w7']] = pr_train[:,0,6].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w8_1']] = pr_train[:,0,7].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w8_2']] = pr_train[:,0,8].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w8_3']] = pr_train[:,0,9].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w8_4']] = pr_train[:,0,10].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w9']] = pr_train[:,0,11].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w10']] = pr_train[:,0,12].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w11']] = pr_train[:,0,13].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w12_1']] = pr_train[:,0,14].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w12_2']] = pr_train[:,0,15].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w12_3']] = pr_train[:,0,16].reshape(-1, 1)
hydrodata.loc[train_dates, ['flow_pred_w12_4']] = pr_train[:,0,17].reshape(-1, 1)
##Training part
hydrodata.to_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Parallel_mse_loss_training.csv')

In [ ]:
hydrodata.loc[test_dates, ['flow_pred_w1']] = pr_test[:,0,0].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w2']] = pr_test[:,0,1].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w3']] = pr_test[:,0,2].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w4']] = pr_test[:,0,3].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w5']] = pr_test[:,0,4].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w6']] = pr_test[:,0,5].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w7']] = pr_test[:,0,6].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w8_1']] = pr_test[:,0,7].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w8_2']] = pr_test[:,0,8].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w8_3']] = pr_test[:,0,9].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w8_4']] = pr_test[:,0,10].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w9']] = pr_test[:,0,11].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w10']] = pr_test[:,0,12].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w11']] = pr_test[:,0,13].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w12_1']] = pr_test[:,0,14].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w12_2']] = pr_test[:,0,15].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w12_3']] = pr_test[:,0,16].reshape(-1, 1)
hydrodata.loc[test_dates, ['flow_pred_w12_4']] = pr_test[:,0,17].reshape(-1, 1)
##Training part + Testing part
##Which is the final out puts
hydrodata.to_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Parallel_mse_loss.csv')


In [ ]:
###select the extraction windows

df = pd.read_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\give\4-inputs_mse\4-inputs_MSE_LOSS_Training.csv')
df.loc[df['Obs_W1'] < 1, 1:] = 0
columns_to_keep = ['Obs_W1', 'Sim_W1', '2*Sim_W1']  
df = df[columns_to_keep]
df.to_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\give\4-inputs_mse\W1_4-inputs_EXTRACTIONS_MSE_Training.csv', index=False)

In [ ]:
##Evaluations
df = pd.read_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\give\4-inputs_mse\W12_4_4-inputs_EXTRACTIONS_MSE_Training.csv')
df2 = pd.read_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\give\4-inputs_mse\W12_4_4-inputs_EXTRACTIONS_MSE_Testing.csv')
# Extract data from two columns
observed_data = df['Obs_W12_4'].values
# simulated_data = df['Sim_W2'].values
simulated_data_2 = df['2*Sim_W12_4'].values


ob_t = df2['Obs_W12_4'].values
sim_t = df2['2*Sim_W12_4'].values

# # Calculate KGE score
# kge_score = calculate_kge(observed_data, simulated_data_2)
# print("KGE score:", kge_score)
NSE_score = cal_nse(observed_data, simulated_data_2)
print("NSE_score:", NSE_score)


Test_NSE = cal_nse(ob_t, sim_t)
print("Test_NSE:", Test_NSE)

In [ ]:
##Plot the results

hydrodata= pd.read_csv(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\scenario3_extractionwindow.csv')
plot_set1 = pd.to_datetime(hydrodata['date'])
#plt.plot(plot_set1, hydrodata['flow_pred_w12_1'],label="simulated_rate", color = '#e41a1c', ls='--')
plt.plot(plot_set1, hydrodata['w1'], label="Observed", color='#377eb8',lw='1.8')
plt.plot(plot_set1, hydrodata['2flow_pred_w1'], label="Simulated", color='#e41a1c', ls='--')
plt.title(f'W1')
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))
plt.gcf().autofmt_xdate()
#plt.ylim([0, 8500])
plt.legend(fontsize = 10)
# plt.text(0.05, 0.95, 'RMSE_Train: 0.684', ha='left', va='top', transform=plt.gca().transAxes)
# plt.text(0.05, 0.85, 'RMSE_Test: 0.882', ha='left', va='top', transform=plt.gca().transAxes)
plt.savefig(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\second\W1.png')
plt.savefig(r'C:\Users\e0427809\Desktop\Try\well_50\results\Plot\second\W1.tiff')
plt.show()